In [1]:
!pip install -q tf-models-official==2.7.0
!pip3 install ktrain

In [5]:
import os
import shutil
import pickle
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
tf.get_logger().setLevel('ERROR')
import ktrain
from ktrain import text

In [6]:
#Load True Articles and Apply true label, remove Reuters header
trueDF = pd.read_csv('True.csv')
trueDF['label'] = 'REAL'
#Common prefixes
twitterParse = '@realDonaldTrump : - '
reutersParse = "(Reuters) - "
#Remove prefixes

trueDF.shape
trueDF.head(10)

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",REAL
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",REAL
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",REAL
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",REAL
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",REAL
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017",REAL
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017",REAL
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017",REAL
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017",REAL
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017",REAL


In [7]:
#Load Fake Articles and Apply fake label
falseDF = pd.read_csv('Fake.csv')
falseDF['label'] = 'FAKE'
falseDF.shape
falseDF.head(10)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",FAKE
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",FAKE
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",FAKE
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",FAKE
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",FAKE
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",FAKE
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",FAKE
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",FAKE
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",FAKE
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",FAKE


In [8]:
#compile information
testDF = pd.concat([trueDF, falseDF])
df = testDF.sample(frac = 1)
df.shape
df.head()

,title,text,subject,date,label
3733,Republicans worry Trump scandals may doom legi...,WASHINGTON (Reuters) - Scandals enveloping U.S...,politicsNews,"May 18, 2017",REAL
640,Trump Loves To Say The New York Times Is ‘Fai...,Print journalism and longstanding papers have ...,News,"August 7, 2017",FAKE
5565,Sessions' role as U.S. attorney general unsett...,"SACRAMENTO, Calif. (Reuters) - The prospect of...",politicsNews,"February 9, 2017",REAL
19202,WATCH PRESIDENT TRUMP ARRIVE ON AIR FORCE ONE:...,The president s visit stretches from Friday af...,left-news,"Feb 3, 2017",FAKE
21246,'Gates of Hell': Iraqi army says fighting near...,(Story corrects third paragraph to show Mosul...,worldnews,"August 29, 2017",REAL


In [9]:
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.3, random_state=7, stratify=df['label'])

In [13]:
(x_train_bert, y_train_bert), (x_val_bert, y_val_bert), preproc = text.texts_from_array(x_train=x_train.to_numpy(), y_train=y_train.to_numpy(),x_test = x_test.to_numpy(), y_test=y_test.to_numpy(),class_names= ['0','1'],preprocess_mode='bert')


preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


C:\Users\adaml\env\HackAI\lib\site-packages\ktrain\utils.py:625: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


task: text classification


In [14]:
with open('trainArr.pkl', 'wb') as handle:
    pickle.dump([x_train_bert, y_train_bert, x_val_bert, y_val_bert], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
with open('trainArr.pkl', 'rb') as handle:
    [x_train_bert, y_train_bert, x_val_bert, y_val_bert] = pickle.load(handle)

In [11]:
model = text.text_classifier('bert', train_data=(x_train_bert, y_train_bert), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train_bert, y_train_bert), val_data=(x_val_bert, y_val_bert), batch_size=8)

Is Multi-Label? False
maxlen is 400
done.


In [ ]:
#learner.lr_find(max_epochs=4)

In [ ]:
#learner.lr_plot()

In [12]:
learner.autofit(10e-2)

early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.1...
Epoch 1/1024
   4/3929 [..............................] - ETA: 7:59:06 - loss: 8.4394 - accuracy: 0.4688

KeyboardInterrupt: 

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
test_df.head()